In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import numpy as np
from models.densenet import DenseNet121
from segment_models.FCN import FCN8s, FCN32s, FCN_mini

In [2]:
from keras import backend as K

Using TensorFlow backend.


In [3]:
smooth = 1.
# dice loss
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_np(y_true,y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [4]:
train_data_path = './data/out/trainImages.npy'
train_mask_path = './data/out/trainMasks.npy'
test_data_path = './data/out/testImages.npy'

train_data = np.load(train_data_path).astype(np.float32)
train_mask = np.load(train_mask_path).astype(np.float32)

train_x = Variable(torch.from_numpy(train_data))
train_y = Variable(torch.from_numpy(train_mask))

In [5]:
rescale = True

if rescale:
    train_data = np.resize(train_data, [train_data.shape[0], train_data.shape[1], 16, 16])
    train_x = Variable(torch.from_numpy(train_data))
    train_mask = np.resize(train_mask, [train_mask.shape[0], train_mask.shape[1], 16, 16])
    train_y = Variable(torch.from_numpy(train_mask))

In [6]:
# net = DenseNet121()
net = FCN_mini()
net.zero_grad()
print('** Init finished **')

** Init finished **


In [7]:
output = net(train_x)
print('** First output finished **')

** First output finished **


/Users/rick/anaconda/envs/py3/lib/python3.5/site-packages/torch/nn/modules/upsampling.py:180: UserWarning: nn.UpsamplingBilinear2d is deprecated. Use nn.Upsample instead.
  warnings.warn("nn.UpsamplingBilinear2d is deprecated. Use nn.Upsample instead.")


In [8]:
loss = dice_coef_loss(output.data.numpy(), train_y.data.numpy())
print(loss)

Tensor("Neg:0", shape=(), dtype=float32)


In [9]:
output.shape

torch.Size([161, 1, 16, 16])

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`